In [5]:
import pandas as pd
import re
from tqdm import tqdm
from sklearn.metrics import f1_score, cohen_kappa_score

In [6]:
def map_fn(row):
    row = str(row)
    if(re.search('yes', row) and re.search('no', row)):
        return None
    elif(re.search('yes', row)):
        return 1
    elif(re.search('no', row)):
        return 0
    elif(re.search('hateful', row)):
        return 1
    elif(re.search('hate.', row)):
        return 1

    return None

In [7]:
files = ['P_base']*6 + ['P'+str(i) for i in range(1,6)]
col_gold = ['isHate'] + 2*['United_States_Hate', 'Australia_Hate', 'United_Kingdom_Hate', 'South_Africa_Hate', 'Singapore_Hate']

type = ['OS_LLMs', 'ChatGPT']
for t in tqdm(type):
    for f,cg in zip(files, col_gold):
        df = pd.read_excel('../'+t+'/'+f+'.xlsx')

        # For FlanT5-XXL, you can choose the below 500 samples and print at the end to check it's performance to compare with ChatGPT
        # Also, keep `files = ['P_base']`, `col_gold = ['isHate']` and `type = ['OS_LLMs']`
        # df = df.sample(n=500, random_state=42).reset_index(drop=True)
        col = df.columns[8:]
        
        no_ans = []
        F1 = []
        IAA = []
        
        for i in col:
            #print(i)
            df[i] = df[i].apply(lambda x: str(x).lower())
            df[i] = df[i].apply(map_fn)
    
            pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
            no_ans.append(df[i].isna().sum())
            
            tmp = df[[cg, i]].dropna()
    
            F1.append(pen*f1_score(tmp[cg], tmp[i], average='weighted'))
        
            IAA.append(pen*cohen_kappa_score(tmp[cg], tmp[i]))
    
        
        res = pd.read_csv('Performance'+t+'.csv')
        try:
            res[f+' '+cg+' Hal'] = no_ans
            res[f+' '+cg+' F1'] = F1
            res[f+' '+cg+' IAA'] = IAA
        except:
            # if there is no output for 8th model (Llama3) then it should append `None`
            no_ans.append(None)
            F1.append(None)
            IAA.append(None)
            res[f+' '+cg+' Hal'] = no_ans
            res[f+' '+cg+' F1'] = F1
            res[f+' '+cg+' IAA'] = IAA

        res.to_csv('Performance'+t+'.csv', index=False)

        # To print for a single compair (FlanT5-XXL -- in case)
        # print(no_ans[-1], F1[-1], IAA[-1])

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:07<00:00,  3.75s/it]
